# Development of machine learning models to process Electronic Health Records – Explainable Models

### Imputation Comparison

Lok Hang Toby Lee (2431180L)

In [172]:
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import seaborn as sn
from sklearn import metrics


# SET YOUR PATH FOR RESOURCES FILE HERE
resources_path = "C:/Users/USER/OneDrive/GU/Year 4/FYP\MIMIC-III-ML/data/resources"
data_path = "C:/Users/USER/OneDrive/GU/Year 4/FYP/MIMIC-III-ML/data/raw"
data = pd.read_csv(data_path+'/mimic_timeseries_data_not_imputed.csv')


In [174]:
# Read in the data and only select the continuous variables:
config = json.load(open(resources_path + '/discretizer_config.json', 'r'))
is_categorical = config['is_categorical_channel']
categorical_var = [key for key, value in config['is_categorical_channel'].items() if value][1:]
patient_identifiers = ['icustay_id', 'subject_id', 'hadm_id', 'hours_in']
data = pd.read_csv(data_path+'/mimic_timeseries_data_not_imputed.csv')

# Filter out subjects with length of stay < 48 hours for the in-hospital mortality task:
y = pd.read_hdf(data_path+'/vitals_hourly_data_preprocessed.h5', 'Y')['los']
indices_to_remove = []
for i, row in y.iteritems():
    if row < 48:
        indices_to_remove.append(i)
        
data = data.reset_index().set_index('icustay_id').drop(indices_to_remove, axis = 0)
y = y.drop(indices_to_remove, axis = 0)

# Extract only the first 48 hours:
data = data[data['hours_in'] < 48].reset_index().set_index(['icustay_id', 'subject_id', 'hadm_id', 'hours_in']).drop('index', axis = 1)

In [175]:
data

Unnamed: 0  Diastolic blood pressure  \
icustay_id subject_id hadm_id hours_in                                         
200003     27513      163557  0                  0                 49.000000   
                              1                  1                 52.000000   
                              2                  2                 52.333333   
                              3                  3                 60.500000   
                              4                  4                 61.000000   
...                                            ...                       ...   
299995     28775      134959  43           2162994                 82.000000   
                              44           2162995                       NaN   
                              45           2162996                       NaN   
                              46           2162997                 65.000000   
                              47           2162998                       NaN   

                                        Fraction inspired oxygen  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                              NaN   
                              1                              NaN   
                              2                              NaN   
                              3                              NaN   
                              4                              NaN   
...                                                          ...   
299995     28775      134959  43                             NaN   
                              44                             NaN   
                              45                             NaN   
                              46                             NaN   
                              47                             NaN   

                                        Glascow coma scale eye opening  \
icustay_id subject_id hadm_id hours_in                                   
200003     27513      163557  0                                    4.0   
                              1                                    NaN   
                              2                                    NaN   
                              3                                    NaN   
                              4                                    NaN   
...                                                                ...   
299995     28775      134959  43                                   NaN   
                              44                                   NaN   
                              45                                   NaN   
                              46                                   4.0   
                              47                                   NaN   

                                        Glascow coma scale motor response  \
icustay_id subject_id hadm_id hours_in                                      
200003     27513      163557  0                                       6.0   
                              1                                       NaN   
                              2                                       NaN   
                              3                                       NaN   
                              4                                       NaN   
...                                                                   ...   
299995     28775      134959  43                                      NaN   
                              44                                      NaN   
                              45                                      NaN   
                              46                                      6.0   
                              47                                      NaN   

                                        Glascow coma scale total  \
icustay_id subject_id hadm_id hours_in                             
200003     27

In [181]:
mean_imputed=pd.read_csv(data_path+'/imputed_data/mean_imputed.csv')
joint_imputed=pd.read_csv(data_path+'/imputed_data/joint_imputed.csv')
mean_imputed = mean_imputed.reset_index().set_index(['icustay_id', 'subject_id', 'hadm_id', 'hours_in']).drop('index', axis = 1)
joint_imputed = joint_imputed.reset_index().set_index(['icustay_id', 'subject_id', 'hadm_id', 'hours_in']).drop('index', axis = 1)


In [185]:
mean_imputed

Diastolic blood pressure  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                        49.000000   
                              1                        52.000000   
                              2                        52.333333   
                              3                        60.500000   
                              4                        61.000000   
...                                                          ...   
299995     28775      134959  43                       82.000000   
                              44                       60.109355   
                              45                       60.109355   
                              46                       65.000000   
                              47                       60.109355   

                                        Fraction inspired oxygen  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                         0.546215   
                              1                         0.546215   
                              2                         0.546215   
                              3                         0.546215   
                              4                         0.546215   
...                                                          ...   
299995     28775      134959  43                        0.546321   
                              44                        0.546321   
                              45                        0.546321   
                              46                        0.546321   
                              47                        0.546321   

                                        Glascow coma scale eye opening  \
icustay_id subject_id hadm_id hours_in                                   
200003     27513      163557  0                                    4.0   
                              1                                    3.0   
                              2                                    3.0   
                              3                                    3.0   
                              4                                    3.0   
...                                                                ...   
299995     28775      134959  43                                   3.0   
                              44                                   3.0   
                              45                                   3.0   
                              46                                   4.0   
                              47                                   3.0   

                                        Glascow coma scale motor response  \
icustay_id subject_id hadm_id hours_in                                      
200003     27513      163557  0                                       6.0   
                              1                                       5.0   
                              2                                       5.0   
                              3                                       5.0   
                              4                                       5.0   
...                                                                   ...   
299995     28775      134959  43                                      5.0   
                              44                                      5.0   
                              45                                      5.0   
                              46                                      6.0   
                              47                                      5.0   

                                        Glascow coma scale total  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                             15.0   
                              1                             12.0   
                              2  

In [186]:
joint_imputed

Diastolic blood pressure  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                        49.000000   
                              1                        52.000000   
                              2                        52.333333   
                              3                        60.500000   
                              4                        61.000000   
...                                                          ...   
299995     28775      134959  43                       82.000000   
                              44                       60.096825   
                              45                       60.096825   
                              46                       65.000000   
                              47                       60.096825   

                                        Fraction inspired oxygen  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                         0.545613   
                              1                         0.545613   
                              2                         0.545613   
                              3                         0.545613   
                              4                         0.545613   
...                                                          ...   
299995     28775      134959  43                        0.545761   
                              44                        0.545761   
                              45                        0.545761   
                              46                        0.545761   
                              47                        0.545761   

                                        Glascow coma scale eye opening  \
icustay_id subject_id hadm_id hours_in                                   
200003     27513      163557  0                                    4.0   
                              1                                    3.0   
                              2                                    3.0   
                              3                                    3.0   
                              4                                    3.0   
...                                                                ...   
299995     28775      134959  43                                   3.0   
                              44                                   3.0   
                              45                                   3.0   
                              46                                   4.0   
                              47                                   3.0   

                                        Glascow coma scale motor response  \
icustay_id subject_id hadm_id hours_in                                      
200003     27513      163557  0                                       6.0   
                              1                                       5.0   
                              2                                       5.0   
                              3                                       5.0   
                              4                                       5.0   
...                                                                   ...   
299995     28775      134959  43                                      5.0   
                              44                                      5.0   
                              45                                      5.0   
                              46                                      6.0   
                              47                                      5.0   

                                        Glascow coma scale total  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                             15.0   
                              1                             12.0   
                              2  

# KNN

Scale data for KNN

In [129]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(data)
scaled_data= scaler.transform(data)
scaled_data=scaled_data[:1000,:]


KNN imputation


In [ ]:
from sklearn.impute import KNNImputer

results_knn_imp = pd.DataFrame(index = data.index, columns = data.columns)

# KNN imputation
knn_imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
knn_imputer.fit(scaled_data)
print("data fitted")
knn_imp = knn_imputer.transform(scaled_data)
print(knn_imp)

Reverse scaling for comparison

In [ ]:
knn_imp=scaler.inverse_transform(knn_imp)
results_knn_imp.loc[:len(knn_imp),:]=knn_imp

In [ ]:
results_knn_imp

Unnamed: 0 Diastolic blood pressure  \
icustay_id subject_id hadm_id hours_in                                       
200003     27513      163557  0               0.0                     49.0   
                              1               1.0                     52.0   
                              2               2.0                52.333333   
                              3               3.0                     60.5   
                              4               4.0                     61.0   
...                                           ...                      ...   
299995     28775      134959  43              NaN                      NaN   
                              44              NaN                      NaN   
                              45              NaN                      NaN   
                              46              NaN                      NaN   
                              47              NaN                      NaN   

                                       Fraction inspired oxygen  \
icustay_id subject_id hadm_id hours_in                            
200003     27513      163557  0                            0.84   
                              1                            0.71   
                              2                            0.71   
                              3                            0.76   
                              4                             0.8   
...                                                         ...   
299995     28775      134959  43                            NaN   
                              44                            NaN   
                              45                            NaN   
                              46                            NaN   
                              47                            NaN   

                                       Glascow coma scale eye opening  \
icustay_id subject_id hadm_id hours_in                                  
200003     27513      163557  0                                   4.0   
                              1                                   3.0   
                              2                                   3.0   
                              3                                   3.0   
                              4                                   3.0   
...                                                               ...   
299995     28775      134959  43                                  NaN   
                              44                                  NaN   
                              45                                  NaN   
                              46                                  NaN   
                              47                                  NaN   

                                       Glascow coma scale motor response  \
icustay_id subject_id hadm_id hours_in                                     
200003     27513      163557  0                                      6.0   
                              1                                      5.6   
                              2                                      5.6   
                              3                                      5.6   
                              4                                      5.6   
...                                                                  ...   
299995     28775      134959  43                                     NaN   
                              44                                     NaN   
                              45                                     NaN   
                              46                                     NaN   
                              47                                     NaN   

                                       Glascow coma scale total  \
icustay_id subject_id hadm_id hours_in                            
200003     27513      163557  0                            15.0   
            

# MICE

In [201]:
import miceforest as mf

mf.ampute_data(data,perc=0.25,random_state=1991)

# Create kernels. 
kernel = mf.ImputationKernel(
  data=data,
  save_all_iterations=True,
  random_state=1991
)

# Run the MICE algorithm for 3 iterations on each of the datasets
kernel.mice(3,verbose=True)

Dataset 0
1  | Heart Rate | Respiratory rate | Systolic blood pressure | Diastolic blood pressure | Mean blood pressure | Oxygen saturation | Weight | Temperature | Glascow coma scale eye opening | Glascow coma scale verbal response | Glascow coma scale motor response | Glucose | Glascow coma scale total | pH | Fraction inspired oxygen | Height
2  | Heart Rate | Respiratory rate | Systolic blood pressure | Diastolic blood pressure | Mean blood pressure | Oxygen saturation | Weight | Temperature | Glascow coma scale eye opening | Glascow coma scale verbal response | Glascow coma scale motor response | Glucose | Glascow coma scale total | pH | Fraction inspired oxygen | Height
3  | Heart Rate | Respiratory rate | Systolic blood pressure | Diastolic blood pressure | Mean blood pressure | Oxygen saturation | Weight | Temperature | Glascow coma scale eye opening | Glascow coma scale verbal response | Glascow coma scale motor response | Glucose | Glascow coma scale total | pH | Fraction insp

In [222]:
mice_imputed = kernel.complete_data(4)
mice_imputed.to_csv(data_path+'/imputed_data/mice_imputed.csv')

In [223]:
mice_imputed

Diastolic blood pressure  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                        49.000000   
                              1                        52.000000   
                              2                        52.333333   
                              3                        60.500000   
                              4                        61.000000   
...                                                          ...   
299995     28775      134959  43                       82.000000   
                              44                       53.000000   
                              45                       96.000000   
                              46                       65.000000   
                              47                       54.000000   

                                        Fraction inspired oxygen  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                             1.00   
                              1                             0.40   
                              2                             0.40   
                              3                             0.50   
                              4                             0.50   
...                                                          ...   
299995     28775      134959  43                            0.70   
                              44                            0.30   
                              45                            0.50   
                              46                            0.28   
                              47                            0.40   

                                        Glascow coma scale eye opening  \
icustay_id subject_id hadm_id hours_in                                   
200003     27513      163557  0                                    4.0   
                              1                                    1.0   
                              2                                    4.0   
                              3                                    4.0   
                              4                                    2.0   
...                                                                ...   
299995     28775      134959  43                                   4.0   
                              44                                   3.0   
                              45                                   2.0   
                              46                                   4.0   
                              47                                   1.0   

                                        Glascow coma scale motor response  \
icustay_id subject_id hadm_id hours_in                                      
200003     27513      163557  0                                       6.0   
                              1                                       4.0   
                              2                                       6.0   
                              3                                       6.0   
                              4                                       5.0   
...                                                                   ...   
299995     28775      134959  43                                      6.0   
                              44                                      6.0   
                              45                                      4.0   
                              46                                      6.0   
                              47                                      4.0   

                                        Glascow coma scale total  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                             15.0   
                              1                              6.0   
                              2  

In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
 
 
# Creating dataset
fig = plt.figure(figsize =(10, 7))
 
# Creating plot
plt.boxplot(joint_imputed_data)
 
# show plot
plt.show()